In [ ]:


import pandas as pd
from rapidfuzz import process, fuzz
import re


file_path = '/content/drive/MyDrive/Colab Notebooks/Axion_Task2/Task 1.xlsx'
excel_data = pd.ExcelFile(file_path)


task_data = excel_data.parse('Task')
taxonomy_data = excel_data.parse('Taxonomy')


def clean_text(text):

    if pd.isna(text):
        return ''
    text = re.sub(r'[^\w\s]', '', str(text))
    text = re.sub(r'\s+', ' ', text)
    return text.strip().lower()


taxonomy_columns = ['Root Cause', 'Symptom Condition ', 'Symptom Component', 'Fix Condition', 'Fix Component']
taxonomy_cleaned = {col: taxonomy_data[col].dropna().apply(clean_text).tolist() for col in taxonomy_columns}


task_data['Complaint'] = task_data['Complaint'].apply(clean_text)
task_data['Cause'] = task_data['Cause'].apply(clean_text)
task_data['Correction'] = task_data['Correction'].apply(clean_text)



for col in ['Root Cause', 'Symptom Condition 1', 'Symptom Component 1', 'Symptom Condition 2', 'Symptom Component 2',
            'Symptom Condition 3', 'Symptom Component 3', 'Fix Condition 1', 'Fix Component 1', 'Fix Condition 2',
            'Fix Component 2', 'Fix Condition 3', 'Fix Component 3']:
    if col not in task_data.columns:
        task_data[col] = ''


def fuzzy_match(text, options, score_cutoff=80):

    matches = process.extract(text, options, scorer=fuzz.partial_ratio, score_cutoff=score_cutoff)
    return [m[0] for m in matches] if matches else []


def tag_data(row):
    complaint = row['Complaint']
    cause = row['Cause']
    correction = row['Correction']


    root_cause = fuzzy_match(complaint, taxonomy_cleaned['Root Cause']) + fuzzy_match(cause, taxonomy_cleaned['Root Cause'])
    symptom_conditions = fuzzy_match(complaint, taxonomy_cleaned['Symptom Condition ']) + fuzzy_match(cause, taxonomy_cleaned['Symptom Condition '])
    symptom_components = fuzzy_match(complaint, taxonomy_cleaned['Symptom Component']) + fuzzy_match(cause, taxonomy_cleaned['Symptom Component'])
    fix_conditions = fuzzy_match(correction, taxonomy_cleaned['Fix Condition'])
    fix_components = fuzzy_match(correction, taxonomy_cleaned['Fix Component'])


    row['Root Cause'] = root_cause[0] if root_cause else ''
    row['Symptom Condition 1'] = symptom_conditions[0] if len(symptom_conditions) > 0 else ''
    row['Symptom Condition 2'] = symptom_conditions[1] if len(symptom_conditions) > 1 else ''
    row['Symptom Condition 3'] = symptom_conditions[2] if len(symptom_conditions) > 2 else ''
    row['Symptom Component 1'] = symptom_components[0] if len(symptom_components) > 0 else ''
    row['Symptom Component 2'] = symptom_components[1] if len(symptom_components) > 1 else ''
    row['Symptom Component 3'] = symptom_components[2] if len(symptom_components) > 2 else ''
    row['Fix Condition 1'] = fix_conditions[0] if len(fix_conditions) > 0 else ''
    row['Fix Condition 2'] = fix_conditions[1] if len(fix_conditions) > 1 else ''
    row['Fix Condition 3'] = fix_conditions[2] if len(fix_conditions) > 2 else ''
    row['Fix Component 1'] = fix_components[0] if len(fix_components) > 0 else ''
    row['Fix Component 2'] = fix_components[1] if len(fix_components) > 1 else ''
    row['Fix Component 3'] = fix_components[2] if len(fix_components) > 2 else ''

    return row


task_data = task_data.apply(tag_data, axis=1)

task_data

,Primary Key,Order Date,Product Category,Complaint,Cause,Correction,Root Cause,Symptom Condition 1,Symptom Component 1,Symptom Condition 2,Symptom Component 2,Symptom Condition 3,Symptom Component 3,Fix Condition 1,Fix Component 1,Fix Condition 2,Fix Component 2,Fix Condition 3,Fix Component 3
0,SO0026296-1,2023-03-08,SPRAYS,visibly notice fasteners under cab on p clips ...,not tighten at factory,go through and retighten all p clips nuts and ...,not tighten,loose,,,,,,retightened,bolts,,bulkhead connector,,vector
1,SO0026385-1,2023-03-08,SPRAYS,fuel door will not stay open,gas strut not installed or anywhere on machine,found gas strut not installed or anywhere on m...,not installed,open,fuel door,wont stay open,,,,installed,gas strut,,,,
2,SO0026385-11,2023-03-08,SPRAYS,compressor pressure line braided steel crushed,compressor pressure line braided steel crushed...,drain air from systemremove associated p clips...,,crushed,compressor pressure line,crushed,braided steel,,compressor pressure line,replaced,braided steel,,compressor line,,
3,SO0028352-1,2023-03-08,SPRAYS,oil running from bottom of machine,oil return under machine swivel fitting left l...,oil return under machine swivel fitting left l...,,oil running,,loose,,,,replaced,oring,,hydraulic,,oring
4,SO0028770-1,2023-03-08,SPRAYS,missing vector intrip unlocks,missing vector intrip unlocks were not installed,installed missing unlocks ran and tested,not installed,missing,vector,missing,intrip unlocks,,vector,installed,,reseted,,,
5,SO0029596-1,2023-03-08,SPRAYS,oil dripping from coupler of return line to pr...,coupler was leaking,remove coupler free up with hammer and socket ...,leaking,oil dripping,coupler,leak,coupler,,,replaced,coupler,,,,
6,SO0058466-2,2023-05-05,SPRAYS,components missing on boom to mount smv sign,not included from factory,service call 7 milesinstall missing brackets a...,not included,missing,boom,,mount svm sign,,,,brackets,,bolts,,svm sign
7,SO0058466-3,2023-05-16,SPRAYS,oil dripping from bottom of machinepictures in...,oring on male quick connect sticking out of fi...,service call14 milesoring on male quick connec...,not included,oil dripping,,,,,,,oring,,hydraulic,,oring
8,SO0058466-4,2023-05-16,SPRAYS,oil leak,blown oring,loaded truck and drove out to sprayer got to s...,blown,oil leak,,leak,,hydraulic leak,,,oring,,oring,,coupler
9,SO0058466-5,2023-05-16,SPRAYS,harness broke,poor material in harness,i was out to farm and i replaced the ncv harne...,poor material,broke,harness,,harness,,,installed,ncv harness,replaced,harness,,


In [ ]:

with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    task_data.to_excel(writer, sheet_name='Tagged_data', index=False)